In [0]:
sales=spark.read.option("header",True).csv("/FileStore/tables/sales.csv")
menu=spark.read.option("header",True).csv("/FileStore/tables/menu.csv")
members=spark.read.option("header",True).csv("/FileStore/tables/members.csv")

In [0]:
menu.printSchema()
from pyspark.sql.types import IntegerType
menu=menu.select(menu.product_id.cast(IntegerType()), menu.product_name, menu.price.cast(IntegerType()), )
menu.printSchema()


root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: integer (nullable = true)

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: integer (nullable = true)



In [0]:
total_amount=sales.join(menu, menu.product_id==sales.product_id,'left').groupBy("customer_id").agg({'price':'sum'}).withColumnRenamed('sum(price)','total_amount').orderBy("customer_id")
total_amount.show()

+-----------+------------+
|customer_id|total_amount|
+-----------+------------+
|          A|          76|
|          B|          74|
|          C|          36|
+-----------+------------+



In [0]:
from pyspark.sql.functions import countDistinct
visited=sales.groupBy("customer_id").agg(countDistinct("order_date"))
visited.show()

+-----------+-----------------+
|customer_id|count(order_date)|
+-----------+-----------------+
|          B|                6|
|          C|                2|
|          A|                4|
+-----------+-----------------+



In [0]:
from pyspark.sql.functions import dense_rank
from pyspark.sql.window import Window
first_item=sales.withColumn("first", dense_rank().over(Window.partitionBy("customer_id").orderBy("order_date"))).filter("first==1").join(menu, menu.product_id==sales.product_id,'left').select('customer_id','product_name').distinct()
first_item.show()

+-----------+------------+
|customer_id|product_name|
+-----------+------------+
|          A|       sushi|
|          A|       curry|
|          B|       curry|
|          C|       ramen|
+-----------+------------+



In [0]:
from pyspark.sql.functions import sum, count,max
most_purchased=sales.join(menu,"product_id").groupBy("product_id","product_name").agg(count("product_id").alias("total_count")).orderBy("total_count",ascending=0).drop("product_id").limit(1)
most_purchased.show()

+------------+-----------+
|product_name|total_count|
+------------+-----------+
|       ramen|          8|
+------------+-----------+



In [0]:
member_first=sales.join(members,"customer_id","inner").filter("order_date>=join_date").withColumn("first",dense_rank().over(Window.partitionBy("customer_id").orderBy("order_date"))).filter("first==1").join(menu,menu.product_id==sales.product_id,"inner").select("customer_id","product_name")
member_first.show()

+-----------+------------+
|customer_id|product_name|
+-----------+------------+
|          A|       curry|
|          B|       sushi|
+-----------+------------+



In [0]:
before_member_first=sales.join(members,"customer_id","inner").filter("order_date<join_date").withColumn("first",dense_rank().over(Window.partitionBy("customer_id").orderBy("order_date"))).filter("first==1").join(menu,menu.product_id==sales.product_id,"inner").select("customer_id","product_name")
member_first.show()

+-----------+------------+
|customer_id|product_name|
+-----------+------------+
|          A|       sushi|
|          A|       curry|
|          B|       curry|
+-----------+------------+

